In [3]:
import pandas as pd

LABEL = "price_doc"

def extract_main_dataset():
    global LABEL
      
    drop_id = True
    drop_cols = []
    
    if drop_id:
        drop_cols.append('id')
    
    train_csv = "/mnt/h/Kaggle/Competitions/Russian Bank/data/train.csv/train.csv"
    test_csv = "/mnt/h/Kaggle/Competitions/Russian Bank/data/test.csv/test.csv"
    macro_csv = "/mnt/h/Kaggle/Competitions/Russian Bank/data/macro.csv/macro.csv"
    fixup_file = "/mnt/h/Kaggle/Competitions/Russian Bank/data/BAD_ADDRESS_FIX.xlsx"
    
    train_df = pd.read_csv(train_csv).set_index('id')

    test_df = pd.read_csv(test_csv).set_index('id')
    macro_df = pd.read_csv(macro_csv)
    
    df_fixup = pd.read_excel(fixup_file)
    df_fixup.set_index('id', inplace=True)
    
    fix_train_indexset = set(df_fixup[df_fixup.index < train_df.index.max()].index)
    bad_train_indexset = set(train_df[train_df.kremlin_km == train_df.kremlin_km.min()].index)
    unfixed_set = bad_train_indexset.difference(fix_train_indexset)

    train_df.drop(unfixed_set, inplace=True)

    df = pd.concat([train_df, test_df])

	# There should be a call to do this, I just don't know it ;) 
    # - edit:  update might be able to do it, but I think the columns would have to be NaN'd out before it works.  
    # So these two lines can stay IMO...
    for c in df_fixup.columns:
        df.loc[df_fixup.index, c] = df_fixup[c]
        
    df_fixed = df
    df_fixed.reset_index(inplace=True)
    # Merge in macroeconomy features
    df_fixed = pd.merge(df_fixed, macro_df, on='timestamp')
    df_fixed = df_fixed.drop(['timestamp'], axis=1)

    
    # One-hot encode categorical columns
    df_fixed = pd.get_dummies(df_fixed)
    
    train = df_fixed[df_fixed.price_doc.notnull()]
    test = df_fixed[df_fixed.price_doc.isnull()].drop(LABEL, axis=1)
    
    return train.drop(drop_cols, axis=1), train[LABEL]

In [ ]:
extract_main_dataset()[0]